# Eat Safe, Love

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from pymongo import UpdateOne
from pymongo.errors import BulkWriteError
from decimal import Decimal

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with ` mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print("Databases:", mongo.list_database_names())

Databases: ['admin', 'classDB', 'config', 'local', 'met', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# Review the collections in the 'uk_food' database
print("Collections in 'uk_food':", db.list_collection_names())

Collections in 'uk_food': ['establishments']


In [6]:
establishments = db['establishments']

In [7]:
# review a document in the establishments collection
pprint(establishments.find_one())

{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67857cd291819e0f76959d9a'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289353',
       

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 Greenwich High Road",
    "AddressLine2": "",
    "AddressLine3": "",
    "AddressLine4": "Greenwich",
    "PostCode": "SE10 8JA",
    "RatingValue": None,
    "geocode": {
        "longitude": 0.0,
        "latitude": 0.0
    },
    "RightToReply": "",
    "Distance": 0.0,
    "NewRatingPending": True
}

In [9]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [10]:
# Check that the new restaurant was inserted
print("Inserted restaurant ID:", result.inserted_id)

Inserted restaurant ID: 67858923d5079920c4366edf


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},  # Filter
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}  # Projection
)

In [12]:
# Display the result
from pprint import pprint
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = 1

# Update the new restaurant with the correct BusinessTypeID
update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"},  # Filter to locate the restaurant
    {"$set": {"BusinessTypeID": business_type_id}}  # Update to set the BusinessTypeID
)

In [14]:
# Confirm that the new restaurant was updated
print("Documents updated:", update_result.modified_count)

Documents updated: 0


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
dover_count

0

In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

print(f"Number of documents deleted: {delete_result.deleted_count}")

Number of documents deleted: 0


In [17]:
# Check if any remaining documents include Dover
remaining_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})

print(f"Remaining documents in Dover: {remaining_dover}")

Remaining documents in Dover: 0


In [18]:
# Check that other documents remain with 'find_one'
sample_document = establishments.find_one()

if sample_document:
    print("Sample document from the remaining collection:")
    from pprint import pprint
    pprint(sample_document)
else:
    print("No documents remain in the collection.")

Sample document from the remaining collection:
{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67857cd291819e0f76959d9a'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'links': [{'href': 'https://api.rati

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Convert latitude and longitude from strings to Decimal numbers using update_many
update_coordinates_result = establishments.update_many(
    {"geocode.latitude": {"$type": "string"}, "geocode.longitude": {"$type": "string"}},
    [
        {"$set": {
            "geocode.latitude": {"$toDecimal": "$geocode.latitude"},
            "geocode.longitude": {"$toDecimal": "$geocode.longitude"}
        }}
    ]
)

# Check that the coordinates are now decimal numbers
sample_document = establishments.find_one()
if sample_document and "geocode" in sample_document:
    print("Sample document with updated coordinates:")
    pprint(sample_document["geocode"])
else:
    print("No document found or coordinates not updated.")


Sample document with updated coordinates:
{'latitude': Decimal128('51.085797'), 'longitude': Decimal128('1.194762')}


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [21]:
# Convert RatingValue from string to integer (if applicable)
update_rating_result = establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

In [22]:
# Check that the coordinates and rating value are now numbers
# Check that the RatingValue is now an integer or null
sample_document_with_rating = establishments.find_one()
if sample_document_with_rating and "RatingValue" in sample_document_with_rating:
    print("Sample document with updated RatingValue:")
    pprint(sample_document_with_rating["RatingValue"])
else:
    print("No document found or RatingValue not updated.")

Sample document with updated RatingValue:
5
